## 1. CandleRule中的相关配置

与"维护实时历史K线功能"重要相关配置为：

|参数|类型|默认值|说明|
|:---|:---|:---|:---|
|INSTIDS|Union[str, list]|'all'|收集的产品 all表示全部正在交易的产品|
|INSTIDS_FILTER|list|[]|过滤的产品|
|INSTID_CONTAINS|str|''|产品名称中需要包含的内容|
|INSTID_ENDSWITH|str|''|产品名称必须以何内容结尾|
|BAR|str|okx.okx_lite.OKX_DEFAULT_BAR|时间颗粒度|
|TIMEZONE|str|okx.okx_lite.OKX_TIMEZONE|时区|
|CANDLE_DIR|str|okx.okx_lite.OKX_CANDLE_DATE_BASE_DIR|以天为单位的数据存储路径|
|LENGTH|int|1440 * 2|candle_map 缓存K线数据长度（实时K线服务）|
|UPDATE_INTERVAL_SECONDS|int|3|candle_map 更新时间间隔（秒）（实时K线服务）|
|CACHE_DELAY_SECONDS|Union[int, None]|60 * 60|candle_map 保存缓存文件时间间隔（秒） None：不保存缓存（实时K线服务）|
|CACHE_DIR|str|'./OKX_CACHE'|缓存数据路径（实时K线服务）|
|LOCAL_CANDLE_DAYS|int|2|服务启动 需维护本地多少天的历史K线数据（实时K线服务）|

## 2. 重新定义规则

In [1]:
from okx.app.candle_server import CandleRule,CandleServer

In [2]:
class MyCandleRule(CandleRule):
    # 收集的产品 all表示全部正在交易的产品
    INSTIDS = [
        'BTC-USD-SWAP',
        'ETH-USD-SWAP',
        'LTC-USD-SWAP',
        'SOL-USD-SWAP',
    ]
    # candle_map 缓存K线数据长度
    LENGTH = 1440
    # 时间颗粒度
    BAR = '1m'
    # 每日下载昨日历史K线数据的时刻 格式：%H:%M:%S None表示不下载
    DOWNLOAD_TIME = None  # 如果你维护的实时K线长度不足以包含一整天，应该关闭DOWNLOAD_TIME功能

## 3. 启动实时K线维护

In [3]:
# 币币交易：SPOT；永续合约：SWAP；交割合约：FUTURES；期权：OPTION
instType = 'SWAP'
# 永续合约，默认规则
candleServer = CandleServer(
    instType=instType,
    rule=MyCandleRule,
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None,
)
# 启动K线维护服务（堵塞直到candle_map维护完成）
candleServer.run_candle_map()
# 被异步维护的candle_map字典
candleServer.candle_map

2024-01-29 22:37:50,275 [INFO] DOWNLOAD BTC-USD-SWAP 1m  2024-01-28
2024-01-29 22:38:00,813 [INFO] DOWNLOAD ETH-USD-SWAP 1m  2024-01-28
2024-01-29 22:38:08,745 [INFO] DOWNLOAD LTC-USD-SWAP 1m  2024-01-28
2024-01-29 22:38:14,682 [INFO] DOWNLOAD SOL-USD-SWAP 1m  2024-01-28
2024-01-29 22:38:14,684 [INFO] COMPLETE DOWNLOAD 2024-01-28 (ALL:4 SKIP:0 SUC:4 WARN:0 ERROR:0)
2024-01-29 22:38:14,687 [INFO] COMPLETE DOWNLOAD 2024-01-27 (ALL:4 SKIP:4 SUC:0 WARN:0 ERROR:0)
2024-01-29 22:38:46,589 [INFO] COMPLETE PREPARE candle_map
2024-01-29 22:38:47,186 [INFO] DOWNLOAD CANDLE CACHE


{'BTC-USD-SWAP': array([[1.70645268e+12, 4.24764000e+04, 4.24833000e+04, ...,
         3.29800000e-01, 1.40000000e+04, 1.00000000e+00],
        [1.70645274e+12, 4.24656000e+04, 4.24719000e+04, ...,
         3.91360000e+00, 1.66200000e+05, 1.00000000e+00],
        [1.70645280e+12, 4.24657000e+04, 4.24738000e+04, ...,
         3.54730000e+00, 1.50600000e+05, 1.00000000e+00],
        ...,
        [1.70653890e+12, 4.18978000e+04, 4.19423000e+04, ...,
         5.04920000e+00, 2.11600000e+05, 1.00000000e+00],
        [1.70653896e+12, 4.19431000e+04, 4.19585000e+04, ...,
         1.34478000e+01, 5.63800000e+05, 1.00000000e+00],
        [1.70653902e+12, 4.18925000e+04, 4.19154000e+04, ...,
         6.87730000e+00, 2.88200000e+05, 1.00000000e+00]]),
 'ETH-USD-SWAP': array([[1.70645268e+12, 2.28101000e+03, 2.28169000e+03, ...,
         7.73420000e+00, 1.76400000e+04, 1.00000000e+00],
        [1.70645274e+12, 2.28089000e+03, 2.28089000e+03, ...,
         9.08545000e+01, 2.07190000e+05, 1.00000000

## 4. 模拟实盘交易实时获取K线

In [4]:
import time
import datetime

# 打印实时更新的candle
for i in range(6):
    # 以安全的方式从candle_map中获取历史线，K线最新时间不能小于当前时间戳大于security_seconds秒
    candle = candleServer.get_candle_security('BTC-USD-SWAP', security_seconds=60 * 3)
    print(
        'K线长度:',
        candle.shape[0],
        'K线的最新时间:',
        datetime.datetime.fromtimestamp(int(candle[-1, 0] / 1000)),
        '当前时间:',
        datetime.datetime.now()
    )
    print(candle)  # 类型：np.ndarray
    time.sleep(10)

K线长度: 1440 K线的最新时间: 2024-01-29 22:37:00 当前时间: 2024-01-29 22:38:47.620350
[[1.70645268e+12 4.24764000e+04 4.24833000e+04 ... 3.29800000e-01
  1.40000000e+04 1.00000000e+00]
 [1.70645274e+12 4.24656000e+04 4.24719000e+04 ... 3.91360000e+00
  1.66200000e+05 1.00000000e+00]
 [1.70645280e+12 4.24657000e+04 4.24738000e+04 ... 3.54730000e+00
  1.50600000e+05 1.00000000e+00]
 ...
 [1.70653890e+12 4.18978000e+04 4.19423000e+04 ... 5.04920000e+00
  2.11600000e+05 1.00000000e+00]
 [1.70653896e+12 4.19431000e+04 4.19585000e+04 ... 1.34478000e+01
  5.63800000e+05 1.00000000e+00]
 [1.70653902e+12 4.18925000e+04 4.19154000e+04 ... 6.87730000e+00
  2.88200000e+05 1.00000000e+00]]
K线长度: 1440 K线的最新时间: 2024-01-29 22:37:00 当前时间: 2024-01-29 22:38:57.624928
[[1.70645268e+12 4.24764000e+04 4.24833000e+04 ... 3.29800000e-01
  1.40000000e+04 1.00000000e+00]
 [1.70645274e+12 4.24656000e+04 4.24719000e+04 ... 3.91360000e+00
  1.66200000e+05 1.00000000e+00]
 [1.70645280e+12 4.24657000e+04 4.24738000e+04 ... 3.547

## 5. 关闭服务

In [5]:
# 关闭服务
candleServer.close_run_candle_map()